## Figures

UDM2 Metadata
https://developers.planet.com/docs/api/udm-2/


Band	Description	Pixel Value Range	Interpretation

Band 1	Clear map	[0, 1]	0: not clear, 1: clear

Band 2	Snow map	[0, 1]	0: no snow or ice, 1: snow or ice

Band 3	Shadow map	[0, 1]	0: no shadow, 1: shadow

Band 4	Light haze map	[0, 1]	0: no light haze, 1: light haze

Band 5	Heavy haze map	[0, 1]	0: no heavy haze, 1: heavy haze

Band 6	Cloud map	[0, 1]	0: no cloud, 1: cloud

Band 7	Confidence map	[0-100]	percentage value: per-pixel algorithmic confidence in classification

Band 8	Unusable pixels	--	Equivalent to the UDM asset: see Planet's Imagery Specification for complete details

In [ ]:
%matplotlib inline
import xarray as xr
import rioxarray
from pyatsa.tseries_preprocess import read_clean_tseries_as_xarr
import matplotlib.pyplot as plt
import numpy as np
import skimage.io as skio

def normalize(v):
    v_min = v.min(axis=(0, 1), keepdims=True)
    v_max = v.max(axis=(0, 1), keepdims=True)
    return (v - v_min)/(v_max - v_min)

def reassign_coords_to_masks(t_series, atsa_masks):
    """
    rioxarray fails to assign the right coordinates sometimes, so we reassign them here based on 
    coordinates from the time series products.
    """
    atsa_masks = atsa_masks.rename({'band':'time'}).assign_coords(time=t_series['reflectance'].time)
    atsa_masks = atsa_masks.assign_coords(y=t_series['reflectance'].y)
    atsa_masks = atsa_masks.assign_coords(x=t_series['reflectance'].x)
    return atsa_masks


def plot_date(t_series, atsa_masks, date_string):
    """
    Plots the results from a pyatsa run and the time series products (reflectance, udm, udm2).
    
    Before running this function, reflectance, udm, and udm2 should be downloaded from Planet
    and placed in a folder. Then call tseries_preprocess.read_clean_tseries_as_xarr() to read in
    the time series products. Run pyatsa_explore.py to generate the ATSA masks. 
    TODO make pyatsa_explore a callable script.
    """
    
    fig, axes = plt.subplots(ncols=3, nrows=2, figsize=(18,12))
    
    xr.plot.imshow(xr.concat([t_series['reflectance'].sel(time=date_string, band=3),
           t_series['reflectance'].sel(time=date_string, band=2),
           t_series['reflectance'].sel(time=date_string, band=1)], 
          dim='band'), robust=True, ax=axes[0,0])
    
    atsa_masks.name = 'ATSA masks'
    
    atsa_masks.sel(time=date_string).plot(ax=axes[0,1])
    
    t_series['udm'].name = 'UDM1'
    
    t_series['udm'].sel(time=date_string, band=1).plot(ax=axes[0,2])
    
    t_series['udm2'].name = 'UDM2 Shadow'
    
    t_series['udm2'].sel(time=date_string, band=3).plot(ax=axes[1,0])
    
    t_series['udm2'].name = 'UDM2 Confidence'
    
    t_series['udm2'].sel(time=date_string, band=7).plot(ax=axes[1,1])
    
    t_series['udm2'].name = 'UDM2 Cloud and Haze'
    
    cloud_and_haze = xr.concat([t_series['udm2'].sel(time=date_string, band=6),
           t_series['udm2'].sel(time=date_string, band=5),
           t_series['udm2'].sel(time=date_string, band=4)], 
          dim='band')
    
    xr.plot.imshow(cloud_and_haze, ax=axes[1,2], robust=True)

    print(np.unique(cloud_and_haze))

In [ ]:
t_series_ag = read_clean_tseries_as_xarr(
    "/home/rave/pyatsa/cfg/buffered_imgs/ag-forestry/")
atsa_masks_ag = xr.open_rasterio("/home/rave/pyatsa/cfg/atsa_results/ag-forestmasks.tif")

In [ ]:
t_series_sav = read_clean_tseries_as_xarr(
    "/home/rave/pyatsa/cfg/buffered_imgs/savanna/")
atsa_masks_sav = xr.open_rasterio("/home/rave/pyatsa/cfg/atsa_results/savanamasks.tif")

In [ ]:
t_series_for = read_clean_tseries_as_xarr(
    "/home/rave/pyatsa/cfg/buffered_imgs/forest/")
atsa_masks_for = xr.open_rasterio("/home/rave/pyatsa/cfg/atsa_results/forestmasks.tif")

In [ ]:
t_series_city = read_clean_tseries_as_xarr(
    "/home/rave/pyatsa/cfg/buffered_imgs/city/")
atsa_masks_city = xr.open_rasterio("/home/rave/pyatsa/cfg/atsa_results/citymasks.tif")

In [ ]:
atsa_masks_ag = reassign_coords_to_masks(t_series_ag, atsa_masks_ag)
atsa_masks_sav = reassign_coords_to_masks(t_series_sav, atsa_masks_sav)
atsa_masks_for = reassign_coords_to_masks(t_series_for, atsa_masks_for)
atsa_masks_city = reassign_coords_to_masks(t_series_city, atsa_masks_city)

In [ ]:
atsa_masks_ag.time

In [ ]:
plot_date(t_series_ag, atsa_masks_ag, '2019-05-26')

Old Figures

In [ ]:
samples = ["forest", "ag-forestry", "city", "savanna"]
f, axarr = plt.subplots(1,4, figsize=(35,10))
for i,sample in enumerate(samples):
    arr = skio.imread("/home/rave/pyatsa/cfg/buffered_chips/scene_number_"+str(24)+"_"+sample+"_.tif")
    axarr[i].imshow(normalize(arr))
    axarr[i].set_title(sample, fontsize=50)
plt.savefig("/home/rave/pyatsa/figures/Figure_1.png")

### 2 - Comparing ATSA, UDM and True Color

In [ ]:
def plot_matching_imgs(scene_id, sample, atsa_masks, udm_masks, udm2_masks, figname):

    atsa = atsa_masks[scene_id]
    udm = udm_masks[scene_id]
    cloud_and_heavy = np.where(udm2_masks[:,:,5,scene_id],4,udm2_masks[:,:,4,scene_id]*3)
    cloud_and_heavy_and_light = np.where(cloud_and_heavy,cloud_and_heavy,udm2_masks[:,:,3,scene_id]*2)
    udm2 = np.where(cloud_and_heavy_and_light, cloud_and_heavy_and_light, udm2_masks[:,:,2,scene_id])
    arr = skio.imread("/home/rave/cloud-free-planet/cfg/buffered_chips/scene_number_"+str(scene_id)+"_"+sample+"_.tif")
    f, axarr = plt.subplots(1,4, figsize=(35,10))
    for i, img in enumerate([(arr, "RGB"), (atsa, "ATSA Result"), (udm, "Usable Data Mask Version 1"), (udm2, "Usable Data Mask Version 2")]):
        if img[0].shape[-1] == 3:
            normed = normalize(img[0])
            axarr[i].imshow(normed)
        else:
            axarr[i].imshow(img[0], "cubehelix")
        axarr[i].set_title(img[1], fontsize=30)
    plt.savefig(figname)

In [ ]:
atsa_masks = skio.imread("/home/rave/cloud-free-planet/cfg/atsa_results/forest_cloud_and_shadow_masks.tif")
udm_masks = skio.imread("/home/rave/cloud-free-planet/cfg/buffered_stacked/forest_udm_stacked.tif")
udm_masks = np.where(udm_masks > 0, 1, 0)
udm2_masks = skio.imread("/home/rave/cloud-free-planet/cfg/buffered_stacked/forest_udm2_stacked.tif")
udm2_masks = np.reshape(udm2_masks, (udm2_masks.shape[0],udm2_masks.shape[1], 8, int(udm2_masks.shape[2]/8)), order='F')


## 0 is RGB, 1 is ATSA Mask, 2 is UDM with all values above 0 unioned, and 3 is UDM2

In [ ]:
plot_matching_imgs(38, "forest", atsa_masks, udm_masks, udm2_masks, "Forest_Results_Cloudy2.png")

In [ ]:
plot_matching_imgs(20, "forest", atsa_masks, udm_masks, udm2_masks)

In [ ]:
### ATSA is masked as all cloud (value of 2)

In [ ]:
plot_matching_imgs(30, "forest", atsa_masks, udm_masks, udm2_masks)

In [ ]:
np.unique(atsa_masks[30])

In [ ]:
atsa_masks = skio.imread("/home/rave/cloud-free-planet/cfg/atsa_results/city_cloud_and_shadow_masks.tif")
udm_masks = skio.imread("/home/rave/cloud-free-planet/cfg/buffered_stacked/city_udm_stacked.tif")
udm_masks = np.where(udm_masks > 0, 1, 0)
udm2_masks = skio.imread("/home/rave/cloud-free-planet/cfg/buffered_stacked/city_udm2_stacked.tif")
udm2_masks = np.reshape(udm2_masks, (udm2_masks.shape[0],udm2_masks.shape[1], 8, int(udm2_masks.shape[2]/8)), order='F')


In [ ]:
plot_matching_imgs(10, "savanna", atsa_masks, udm_masks, udm2_masks, "Savanna_Results_Cloudy.png")

In [ ]:
scene_id = 38
cloud_and_heavy = np.where(udm2_masks[:,:,5,scene_id],4,udm2_masks[:,:,4,scene_id]*3)
cloud_and_heavy_and_light = np.where(cloud_and_heavy,cloud_and_heavy,udm2_masks[:,:,3,scene_id]*2)
udm2 = np.where(cloud_and_heavy_and_light, cloud_and_heavy_and_light, udm2_masks[:,:,2,scene_id])

In [ ]:
np.unique(udm2)

In [ ]:
skio.imshow(udm2)

In [ ]:
plot_matching_imgs(20, "city", atsa_masks, udm_masks, udm2_masks)

In [ ]:
plot_matching_imgs(30, "city", atsa_masks, udm_masks, udm2_masks)

In [ ]:
plot_matching_imgs(29, "city", atsa_masks, udm_masks, udm2_masks)

In [ ]:
plot_matching_imgs(31, "city", atsa_masks, udm_masks, udm2_masks)

In [ ]:
atsa_masks = skio.imread("/home/rave/cloud-free-planet/cfg/atsa_results/savanna_cloud_and_shadow_masks.tif")
udm_masks = skio.imread("/home/rave/cloud-free-planet/cfg/buffered_stacked/savanna_udm_stacked.tif")
udm_masks = np.where(udm_masks > 0, 1, 0)
udm2_masks = skio.imread("/home/rave/cloud-free-planet/cfg/buffered_stacked/savanna_udm2_stacked.tif")
udm2_masks = np.reshape(udm2_masks, (udm2_masks.shape[0],udm2_masks.shape[1], 8, int(udm2_masks.shape[2]/8)), order='F')

In [ ]:
skio.imshow(udm_masks[0])

In [ ]:
plot_matching_imgs(40, "savanna", atsa_masks, udm_masks, udm2_masks, "Savanna_water.png")

In [ ]:
plot_matching_imgs(57, "savanna", atsa_masks, udm_masks, udm2_masks)

In [ ]:
atsa_masks.shape

In [ ]:
np.unique(atsa_masks[20])

In [ ]:
plot_matching_imgs(40, "savanna", atsa_masks, udm_masks, udm2_masks)

In [ ]:
atsa_masks = skio.imread("/home/rave/cloud-free-planet/cfg/atsa_results/ag-forestry_cloud_and_shadow_masks.tif")
udm_masks = skio.imread("/home/rave/cloud-free-planet/cfg/buffered_stacked/ag-forestry_udm_stacked.tif")
udm_masks = np.where(udm_masks > 0, 1, 0)
udm2_masks = skio.imread("/home/rave/cloud-free-planet/cfg/buffered_stacked/ag-forestry_udm2_stacked.tif")
udm2_masks = np.reshape(udm2_masks, (udm2_masks.shape[0],udm2_masks.shape[1], 8, int(udm2_masks.shape[2]/8)), order='F')

In [ ]:
plot_matching_imgs(0, "ag-forestry", atsa_masks, udm_masks, udm2_masks, "Savanna_1.png")

In [ ]:
plot_matching_imgs(10, "ag-forestry", atsa_masks, udm_masks, udm2_masks, "Savanna_2.png")

In [ ]:
plot_matching_imgs(20, "ag-forestry", atsa_masks, udm_masks, udm2_masks, "Savanna_3.png")

In [ ]:
plot_matching_imgs(30, "ag-forestry", atsa_masks, udm_masks, udm2_masks, "Savanna_4.png")

In [ ]:
plot_matching_imgs(40, "ag-forestry", atsa_masks, udm_masks, udm2_masks, "Savanna_5.png")